In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
coins = cv2.cvtColor(cv2.imread('../../data/pennies.jpg'),cv2.COLOR_BGR2RGB)
plt.imshow(coins)


In [ ]:
# blur
blur_pennies =  cv2.medianBlur(coins,25)
# grayscale 
gray_coins = cv2.cvtColor(blur_pennies,cv2.COLOR_RGB2GRAY)
# threshold
res,thresh = cv2.threshold(gray_coins,160,255,cv2.THRESH_BINARY_INV)
plt.imshow(thresh,cmap='gray')


In [ ]:
# Find Contours
contours, heirarchy = cv2.findContours(thresh,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
for i in range(len(contours)):
    if heirarchy[0][i][3] == -1:
        cv2.drawContours(coins, contours, i, (255,0,0), 10)
plt.imshow(coins,cmap="gray")

In [ ]:
# Attempt to erode before finding contours
kernel = np.ones((7,7),dtype=np.uint8)
erode_coins = cv2.erode(thresh,kernel,iterations=6)
plt.imshow(erode_coins)


# Watershed algorithm

In [ ]:
# Read in image
coins = cv2.cvtColor(cv2.imread('../../data/pennies.jpg'),cv2.COLOR_BGR2RGB)

# blur. Large image, 0.8 ~ 1.2 % kernel
blur_pennies =  cv2.medianBlur(coins,35)

# grayscale 
gray_coins = cv2.cvtColor(blur_pennies,cv2.COLOR_RGB2GRAY)
# plt.imshow(gray_coins,cmap='gray')

# threshold (min to max, use otsu in conjunction with binary thresholding)
res,thresh = cv2.threshold(gray_coins,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# noise removal
kernel = np.ones((3,3),np.uint8)
denoised = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel,iterations=2)
# plt.imshow(denoised,cmap='gray')

sure_bg = cv2.dilate(denoised,kernel,iterations=3)
# plt.imshow(sure_bg,cmap='gray')

In [ ]:
# Find seeds from the image, via distance transform.
# distance from 0th value aka background 

dist_transform = cv2.distanceTransform(denoised,cv2.DIST_L2,5)

# Threshold on the points found
ret, sure_fg = cv2.threshold(dist_transform,230,255,cv2.THRESH_BINARY)
plt.imshow(sure_fg,cmap='gray')


In [ ]:
sure_fg = np.uint8(sure_fg)
unknown =  cv2.subtract(denoised,sure_fg)
plt.imshow(unknown,cmap='gray')

In [ ]:
ret,markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown==255] = 0
plt.imshow(markers,cmap='gray')

In [ ]:
watershed = cv2.watershed(coins,markers)
plt.imshow(watershed, cmap='gray')

In [ ]:
contours, heirarchy = cv2.findContours(watershed,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
for i in range(len(contours)):
    if heirarchy[0][i][3] == -1:
        cv2.drawContours(coins, contours, i, (255,0,0), 10)
plt.imshow(coins,cmap="gray")